In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dash
from dash import dcc,html
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input,Output
import plotly.graph_objects as go
from wordcloud import WordCloud
import numpy as np
from PIL import Image
import circlify
url2='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m5_survey_data_technologies_normalised.csv'
df2=pd.read_csv(url2)
fig_size = (8, 6)
ch11=df2.groupby('LanguageWorkedWith',as_index=False)['Respondent'].count().sort_values('Respondent',ascending=False).head(10)
fig_11 = px.bar(ch11, x='Respondent', y='LanguageWorkedWith', color='LanguageWorkedWith', orientation='h',
             labels={'Respondent': 'Count', 'LanguageWorkedWith': 'Language'},
             text='Respondent', title='Top 10 Programming Languages',width=800, height=600)

# Customize text positioning (optional)
fig_11.update_traces(textposition='inside')  # Adjust as needed
fig_11.update_layout(title_x=0.5,showlegend=False)

ch12=df2.groupby('DatabaseWorkedWith',as_index=False)['Respondent'].count().sort_values('Respondent',ascending=False).head(10)
fig_12 = px.bar(ch12, x='DatabaseWorkedWith', y='Respondent', color='DatabaseWorkedWith',
             labels={'DatabaseWorkedWith': 'Data Base','Respondent': 'Count'},
             text='Respondent', title='Top 10 Programming Data Base',width=800, height=600)

# Customize text positioning (optional)
fig_12.update_traces(textposition='outside')  # Adjust as needed
fig_12.update_layout(title_x=0.5,showlegend=False,)

#ch13=df2.groupby('PlatformWorkedWith',as_index=False)['Respondent'].count().sort_values('Respondent',ascending=False)
#fig_13 = px.wordcloud(ch13, x='Respondent', y='LanguageWorkedWith', color='LanguageWorkedWith', orientation='h',
#labels={'Respondent': 'Count', 'LanguageWorkedWith': 'Language'},
             #text='Respondent', title='Top 10 Programming Languages')

# Customize text positioning (optional)
#fig_11.update_traces(textposition='inside')  # Adjust as needed
#fig_11.update_layout(title_x=0.5,showlegend=False)
ch13=df2.groupby('PlatformWorkedWith',as_index=False)['Respondent'].count().sort_values('Respondent',ascending=False)
ch14 = ch13.reset_index().drop('index', axis=1)
my_dict = {}
for i, x in zip(ch14['PlatformWorkedWith'], ch14['Respondent']):
    my_dict[i] = x

# Generate a word cloud
wordcloud = WordCloud(width=800, height=600, background_color='white',colormap='cubehelix').generate_from_frequencies(my_dict)

# Convert wordcloud to an image array
wordcloud_img = wordcloud.to_image()
wordcloud_array = np.array(wordcloud_img)

# Create a plotly figure
fig3 = go.Figure()

# Add image trace to plotly figure
fig3.add_trace(go.Image(z=wordcloud_array))

# Update layout (with proper indentation)
fig3.update_layout(title='Platform Worked With',title_x=0.5,
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    margin=dict(l=20, r=20, t=40, b=20)
)
###
#ch15=df2.groupby('WebFrameWorkedWith',as_index=False)['Respondent'].count().sort_values('Respondent',ascending=False).head(10)
ch15 = df2.groupby('WebFrameWorkedWith')['Respondent'].count().sort_values(ascending=True).tail(10)
# Sort DataFrame by Respondent in descending order
ch15 = ch15.reset_index()
# Normalize the Respondent values
ch15['Normalized'] = ch15['Respondent'] / ch15['Respondent'].max()  

colors = {
    ch15['WebFrameWorkedWith'][0]: 'blue',
    ch15['WebFrameWorkedWith'][1]: 'red',
    ch15['WebFrameWorkedWith'][2]: 'green',
    ch15['WebFrameWorkedWith'][3]: 'purple',
    ch15['WebFrameWorkedWith'][4]: 'orange',
    ch15['WebFrameWorkedWith'][5]: 'brown',
    ch15['WebFrameWorkedWith'][6]: 'pink',
    ch15['WebFrameWorkedWith'][7]: 'yellow',
    ch15['WebFrameWorkedWith'][8]: 'cyan',
    ch15['WebFrameWorkedWith'][9]: 'magenta'
}

# Create circle packing data
circles = circlify.circlify(
    ch15['Normalized'].tolist(), 
    show_enclosure=False, 
    target_enclosure=circlify.Circle(x=0, y=0, r=1)
)

# Create Plotly figure
fig = go.Figure()

# Plot circles
for circle, label in zip(circles, ch15['WebFrameWorkedWith']):
    x, y, r = circle
    fig.add_shape(type="circle",
                  x0=x-r, y0=y-r, x1=x+r, y1=y+r,
                  line=dict(color='black'),
                  fillcolor=colors[label],
                  opacity=0.5)
    fig.add_trace(go.Scatter(
        x=[x], y=[y],
        mode='text',
        text=[label],
        textposition='middle center'
    ))

fig.update_layout(
    title='Hierarchy Bubble Chart of Web Frameworks',
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    xaxis_showticklabels=False,
    yaxis_showticklabels=False,
    autosize=False,
    width=600,
    height=600,
    paper_bgcolor='white',
    plot_bgcolor='white'
)


app = dash.Dash(__name__)

app.layout = html.Div([
                html.Div([
                dcc.Graph(id='bar-chart1', figure=fig_11),
                dcc.Graph(id='bar-chart2', figure=fig_12)],style={'display':'flex'}),
                html.Br(),
                html.Div([
                dcc.Graph(id='bar-chart3', figure=fig3),
                dcc.Graph(id='bar-chart4', figure=fig)],style={'display':'flex'})
                ])
if __name__ == '__main__':
    app.run_server(port=8053,debug=True)